# Benchmarking for kriging equations
---

In [1]:
import numpy as np
import scipy

from sklearn.datasets import make_spd_matrix

## Benchmarking SPD matrix inversion

In [2]:
# establish SPD matrix
def SPD(N):
    X = np.linspace(0, 1, N*N).reshape(N, N)
    X = 0.5*(X + X.T) + np.eye(N) * N
    return X

N = 1000
X1 = make_spd_matrix(N)
X2 = SPD(N)

In [10]:
# default
X_inv_1 = np.linalg.inv(X2)
%timeit np.linalg.inv(X2)

73.1 ms ± 4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
# cholesky numpy
c = np.linalg.inv(np.linalg.cholesky(X2)); X_inv_2 = np.dot(c.T,c)
%timeit c = np.linalg.inv(np.linalg.cholesky(X2));np.dot(c.T,c)

155 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
# cholesky scipy
Ip = np.eye(N)
X_inv_3 = scipy.linalg.cho_solve(scipy.linalg.cho_factor(X2,lower=True), Ip)
%timeit scipy.linalg.cho_solve(scipy.linalg.cho_factor(X2,lower=True), Ip)

52.5 ms ± 796 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
print(np.allclose(X_inv_1, X_inv_2), np.allclose(X_inv_1, X_inv_3))

True True


Conclusion: take the scipy `cho_solve` approach since it's the fastest and will be most stable using the actual data vector rather than computing the entire inverse.

## Benchmark matrix multiplication

In [21]:
# N = 360*120
N = 5000
b = np.random.rand(N, 1)
v = np.random.rand(N, N)

In [22]:
P1 = np.matmul(v, b)
P2 = np.dot(v, b)
np.allclose(P1, P2)

True

In [23]:
%timeit np.matmul(v, b)

46.2 ms ± 8.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit np.dot(v, b)

62.6 ms ± 9.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
